# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Shopping Mall in Kuala Lumpur, Malaysia_**
- Build a dataframe of neighborhoods in Kuala Lumpur, Malaysia by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Import libraries

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [8]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai").text

In [9]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [10]:
# create a list to store neighborhood data
neighborhoodList = []

In [11]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [12]:
# create a new DataFrame from the list
chn_df = pd.DataFrame({"Neighborhood": neighborhoodList})

chn_df.head()

,Neighborhood
0,Alandur
1,Anna Nagar
2,"Ashok Nagar, Chennai"
3,Assisi Nagar
4,Ayanavaram


In [13]:
# print the number of rows of the dataframe
chn_df.shape

(61, 1)

### 3. Get the geographical coordinates

In [14]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, Tamil Nadu'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [15]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in chn_df["Neighborhood"].tolist() ]

In [16]:
coords

[[13.00013000000007, 80.20060000000007],
 [13.083590000000072, 80.21020000000004],
 [13.035390000000064, 80.21220000000005],
 [13.164570000000026, 80.23274000000004],
 [13.09883000000002, 80.23238000000003],
 [13.074621704974275, 80.24277657646144],
 [12.932770000000062, 80.14387000000005],
 [12.95234000000005, 80.14411000000007],
 [12.988610000000051, 80.15100000000007],
 [12.82725000000005, 80.22866000000005],
 [13.040920000000028, 80.13649000000004],
 [13.11035000000004, 80.21301000000005],
 [13.129720000000077, 80.18300000000005],
 [13.120580000000075, 80.06047000000007],
 [12.956150000000036, 80.17885000000007],
 [12.793410000000051, 80.22010000000006],
 [13.081980000000044, 80.24448000000007],
 [13.051520000000039, 80.22421000000008],
 [13.136630000000025, 80.24479000000008],
 [13.131830000000036, 80.19928000000004],
 [13.096050000000048, 80.05292000000009],
 [13.116800000000069, 80.27726000000007],
 [13.183260000000075, 80.24059000000005],
 [13.157520000000034, 80.24283000000008

In [17]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [18]:
# merge the coordinates into the original dataframe
chn_df['Latitude'] = df_coords['Latitude']
chn_df['Longitude'] = df_coords['Longitude']

In [20]:
# check the neighborhoods and the coordinates
print(chn_df.shape)
chn_df

(61, 3)


,Neighborhood,Latitude,Longitude
0,Alandur,13.000130,80.200600
1,Anna Nagar,13.083590,80.210200
2,"Ashok Nagar, Chennai",13.035390,80.212200
3,Assisi Nagar,13.164570,80.232740
4,Ayanavaram,13.098830,80.232380
5,Chennai city,13.074622,80.242777
6,Chitlapakkam,12.932770,80.143870
7,Chromepet,12.952340,80.144110
8,Cowl Bazaar,12.988610,80.151000
9,Egattur (Kanchipuram District),12.827250,80.228660


In [21]:
# save the DataFrame as CSV file
chn_df.to_csv("chn_df.csv", index=False)

# 4. Create a map of Mumbai with neighborhoods superimposed on top

In [22]:
# get the coordinates of Mumbai
address = 'Chennai,Tamil Nadu'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai,Tamil Nadu {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai,Tamil Nadu 13.0801721, 80.2838331.


In [24]:
# create map of Mumbai using latitude and longitude values
map_chn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(chn_df['Latitude'], chn_df['Longitude'], chn_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_chn)  
    
map_mum

In [25]:
# save the map as HTML file
map_chn.save('map_chn.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [52]:
# define Foursquare Credentials and Version
CLIENT_ID = 'XLV5BH1W5W3GGUOQ4PQBGRHKR51WBWAJHBTR4UY3SLW3DG3C' # your Foursquare ID
CLIENT_SECRET = 'KUIBKVNUUOUF0N2ALME4SCXE3C12YI3TMDQUPE5VQQLXFVTP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XLV5BH1W5W3GGUOQ4PQBGRHKR51WBWAJHBTR4UY3SLW3DG3C
CLIENT_SECRET:KUIBKVNUUOUF0N2ALME4SCXE3C12YI3TMDQUPE5VQQLXFVTP


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [53]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(chn_df['Latitude'], chn_df['Longitude'], chn_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [54]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1214, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alandur,13.00013,80.2006,Sukkkubai Beef Biryani Shop,12.998769,80.201381,Indian Restaurant
1,Alandur,13.00013,80.2006,Pizza Republic,12.990987,80.198613,Pizza Place
2,Alandur,13.00013,80.2006,St. Thomas Mount,13.004752,80.193486,Church
3,Alandur,13.00013,80.2006,Moon & Six Pence - The Irish Bar,13.007848,80.208152,Bar
4,Alandur,13.00013,80.2006,Radisson Blu,12.994229,80.187562,Hotel


**Let's check how many venues were returned for each neighorhood**

In [55]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alandur,43,43,43,43,43,43
Anna Nagar,100,100,100,100,100,100
"Ashok Nagar, Chennai",67,67,67,67,67,67
Assisi Nagar,4,4,4,4,4,4
Ayanavaram,31,31,31,31,31,31
Chennai city,100,100,100,100,100,100
Chitlapakkam,13,13,13,13,13,13
Chromepet,20,20,20,20,20,20
Cowl Bazaar,19,19,19,19,19,19


**Let's find out how many unique categories can be curated from all the returned venues**

In [56]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 149 uniques categories.


In [57]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Pizza Place', 'Church', 'Bar', 'Hotel',
       'Restaurant', 'Asian Restaurant', 'Ice Cream Shop',
       'Train Station', 'South Indian Restaurant', 'Donut Shop',
       'Breakfast Spot', "Men's Store", 'Japanese Restaurant',
       'Pool Hall', 'Bakery', 'Italian Restaurant',
       'Multicuisine Indian Restaurant', 'Airport', 'Bus Station', 'Café',
       'Juice Bar', 'Fast Food Restaurant', 'Metro Station', 'Cafeteria',
       'Gym Pool', 'Sandwich Place', 'Park', 'Gym', 'Coffee Shop',
       'Snack Place', 'Chinese Restaurant', 'Indian Sweet Shop',
       'Vegetarian / Vegan Restaurant', 'Shoe Store', 'Burger Joint',
       'Middle Eastern Restaurant', 'American Restaurant',
       'Shopping Mall', 'Department Store', 'Farmers Market',
       'Clothing Store', 'New American Restaurant', 'Multiplex',
       'BBQ Joint', 'Paper / Office Supplies Store', 'Market',
       'Jewelry Store', 'Bookstore', 'Furniture / Home Store'],
      dtype=object)

In [58]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

False

### 6. Analyze Each Neighborhood

In [59]:
# one hot encoding
chn_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chn_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chn_onehot.columns[-1]] + list(chn_onehot.columns[:-1])
chn_onehot = chn_onehot[fixed_columns]

print(chn_onehot.shape)
chn_onehot.head()

(1214, 150)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Big Box Store,Bistro,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Construction & Landscaping,Convenience Store,Coworking Space,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,Hookah Bar,Hospital,Hotel,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,National Park,New American Restaurant,Nightclub,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Supermarket,Taco Place,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [61]:
chn_grouped = chn_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(chn_grouped.shape)
chn_grouped

(61, 150)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Big Box Store,Bistro,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Construction & Landscaping,Convenience Store,Coworking Space,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,Hookah Bar,Hospital,Hotel,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,National Park,New American Restaurant,Nightclub,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Supermarket,Taco Place,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.00,0.000000,0.000000,0.023256,0.00,0.000000,0.000000,0.023256,0.023256,0.000000,0.000000,0.00,0.000000,0.00,0.046512,0.00,0.000000,0.000000,0.023256,0.000000,0.023256,0.046512,0.0,0.000000,0.00,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.023256,0.000000,0.00,0.000000,0.116279,0.000000,0.000000,0.023256,0.162791,0.00,0.000000,0.000000,0.023256,0.00,0.023256,0.00,0.023256,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.023256,0.023256,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.046512,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.069767,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.046512,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.046512,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,Anna Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.010000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.01,0.020000,0.00,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.000000,0.040000,0.0,0.030000,0.00,0.000000,0.040000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.010000,0.010000,0.110000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.020000,0.000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.030000,0.130000,0.01,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.010000,0.00,0.000000,0.00,0.0,0.000000

In [62]:
len(chn_grouped[chn_grouped["Shopping Mall"] > 0])

11

**Create a new DataFrame for Shopping Mall data only**

In [63]:
chn_mall = chn_grouped[["Neighborhoods","Shopping Mall"]]

In [64]:
chn_mall.head()

,Neighborhoods,Shopping Mall
0,Alandur,0.000000
1,Anna Nagar,0.020000
2,"Ashok Nagar, Chennai",0.014925
3,Assisi Nagar,0.000000
4,Ayanavaram,0.032258


# 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [65]:
# set number of clusters
kclusters = 3

chn_clustering = chn_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chn_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 2, 0, 0, 1, 0, 1])

In [66]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
chn_merged = chn_mall.copy()

# add clustering labels
chn_merged["Cluster Labels"] = kmeans.labels_

In [67]:
chn_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
chn_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alandur,0.000000,0
1,Anna Nagar,0.020000,2
2,"Ashok Nagar, Chennai",0.014925,2
3,Assisi Nagar,0.000000,0
4,Ayanavaram,0.032258,2


In [68]:
chn_merged = chn_merged.join(chn_df.set_index("Neighborhood"), on="Neighborhood")

print(chn_merged.shape)
chn_merged.head() # check the last columns!

(61, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.00013,80.20060
1,Anna Nagar,0.020000,2,13.08359,80.21020
2,"Ashok Nagar, Chennai",0.014925,2,13.03539,80.21220
3,Assisi Nagar,0.000000,0,13.16457,80.23274
4,Ayanavaram,0.032258,2,13.09883,80.23238


In [69]:
# sort the results by Cluster Labels
print(chn_merged.shape)
chn_merged.sort_values(["Cluster Labels"], inplace=True)
chn_merged

(61, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.000130,80.200600
31,Navalur,0.000000,0,12.845840,80.226480
32,Nazarethpettai,0.000000,0,13.037100,80.057550
33,Oragadam,0.000000,0,13.137440,80.153830
34,Padappai,0.000000,0,12.886500,80.021550
35,Pallavaram,0.000000,0,12.974440,80.148520
36,Pallikaranai,0.000000,0,12.955670,80.220800
38,Panambakkam,0.000000,0,13.077610,80.155830
39,Pattabiram,0.000000,0,13.123330,80.059440
40,Perambakkam,0.000000,0,12.905630,80.209070


**Finally, let's visualize the resulting clusters**

In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chn_merged['Latitude'], chn_merged['Longitude'], chn_merged['Neighborhood'], chn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [71]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [72]:
chn_merged.loc[chn_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.0,0,13.000130,80.200600
31,Navalur,0.0,0,12.845840,80.226480
32,Nazarethpettai,0.0,0,13.037100,80.057550
33,Oragadam,0.0,0,13.137440,80.153830
34,Padappai,0.0,0,12.886500,80.021550
35,Pallavaram,0.0,0,12.974440,80.148520
36,Pallikaranai,0.0,0,12.955670,80.220800
38,Panambakkam,0.0,0,13.077610,80.155830
39,Pattabiram,0.0,0,13.123330,80.059440
40,Perambakkam,0.0,0,12.905630,80.209070


#### Cluster 1

In [73]:
chn_merged.loc[chn_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
7,Chromepet,0.050000,1,12.95234,80.14411
29,Mugalivakkam,0.047619,1,13.01895,80.16813
9,Egattur (Kanchipuram District),0.052632,1,12.82725,80.22866
37,Pammal,0.045455,1,12.96814,80.13359


#### Cluster 2

In [74]:
chn_merged.loc[chn_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
4,Ayanavaram,0.032258,2,13.09883,80.23238
2,"Ashok Nagar, Chennai",0.014925,2,13.03539,80.21220
1,Anna Nagar,0.020000,2,13.08359,80.21020
57,Vadapalani,0.014493,2,13.05226,80.21120
58,Virugambakkam,0.017241,2,13.05590,80.19349
17,Kodambakkam,0.020000,2,13.05152,80.22421
16,Kilpauk,0.017857,2,13.08198,80.24448


#### Observations:
Most of the shopping malls are concentrated in the central area of Chennai city, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.